In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

In [2]:
df = (
    pd
    .read_csv(
        "../data/interim/data_pre_processados.csv",
    )
    # .drop(columns=["ID"], axis=1)
)

In [3]:
df.head()

,limite,sexo,educacao,estado_civil,idade,status_0509,status_0508,status_0507,status_0506,status_0505,...,saldo_0506,saldo_0505,saldo_0504,pago_0509,pago_0508,pago_0507,pago_0506,pago_0505,pago_0504,mau
0,20000,feminino,universidade,casado,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,sim
1,120000,feminino,universidade,solteiro,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,sim
2,90000,feminino,universidade,solteiro,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,nao
3,50000,feminino,universidade,casado,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,nao
4,50000,masculino,universidade,casado,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,nao


In [4]:
X = df.drop(columns=['mau'], axis=1)
y = df[['mau']]

In [5]:
nominal_columns = ['sexo', 'educacao', 'estado_civil']
numerical_columns = list(set(df.columns) - set(nominal_columns) - {'mau'}) 
# numerical_columns = [
#     coluna for coluna in df.columns if coluna not in nominal_columns and coluna != 'mau'
# ]

In [6]:
preprocessor = ColumnTransformer([
    ("one-hot-encoding", OneHotEncoder(), nominal_columns),
    ("scaler", StandardScaler(), numerical_columns)
])

In [7]:
models = {
    'logistic-regression': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(),
    'gradient-boosting': GradientBoostingClassifier()
}

In [8]:
results = {}
n_splits = 30
for model_name, model in models.items():
    scores = cross_validate(
        estimator = Pipeline(steps=[
            ("preprocessamento", preprocessor),
            ("modelo", model)
        ]),
        X = X,
        y = y.replace({'sim': 1, 'nao': 0}),
        cv = ShuffleSplit(n_splits=n_splits, test_size=.2),
        n_jobs = 2,
        scoring = ['accuracy', 'f1']
    )
    scores['model'] = [model_name] * n_splits
    if not(results):
        results = scores
    else:
        for key in results:
            results[key] = list(results[key])
            results[key].extend(scores[key])

/home/vilquer/.cache/pypoetry/virtualenvs/src-adCLVQQ5-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vilquer/.cache/pypoetry/virtualenvs/src-adCLVQQ5-py3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/vilquer/.cache/pypoetry/virtualenvs/src-adCLVQQ5-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataCon

In [9]:
pd.DataFrame(results).groupby('model').agg([np.mean, np.std]).transpose()

model                    KNN        SVM  gradient-boosting  \
fit_time      mean  0.045007  45.802209           8.957825   
              std   0.019052  19.808549           1.115820   
score_time    mean  3.916671   4.824502           0.022121   
              std   2.184379   2.245913           0.010432   
test_accuracy mean  0.793439   0.819806           0.820211   
              std   0.004699   0.004347           0.004867   
test_f1       mean  0.434014   0.452640           0.473000   
              std   0.010189   0.012331           0.013610   

model               logistic-regression  
fit_time      mean             0.450972  
              std              0.023725  
score_time    mean             0.023317  
              std              0.001447  
test_accuracy mean             0.812944  
              std              0.005301  
test_f1       mean             0.363234  
              std              0.014287

/home/vilquer/.cache/pypoetry/virtualenvs/src-adCLVQQ5-py3.8/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vilquer/.cache/pypoetry/virtualenvs/src-adCLVQQ5-py3.8/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vilquer/.cache/pypoetry/virtualenvs/src-adCLVQQ5-py3.8/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vilquer/.cache/pypoetry/virtualenvs/src-adCLVQQ5-py3.8/lib/python3.8/site

In [10]:
X

,limite,sexo,educacao,estado_civil,idade,status_0509,status_0508,status_0507,status_0506,status_0505,...,saldo_0507,saldo_0506,saldo_0505,saldo_0504,pago_0509,pago_0508,pago_0507,pago_0506,pago_0505,pago_0504
0,20000,feminino,universidade,casado,24,2,2,-1,-1,-2,...,689,0,0,0,0,689,0,0,0,0
1,120000,feminino,universidade,solteiro,26,-1,2,0,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,feminino,universidade,solteiro,34,0,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,feminino,universidade,casado,37,0,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,masculino,universidade,casado,57,-1,0,-1,0,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,masculino,ensino médio,casado,39,0,0,0,0,0,...,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000
29996,150000,masculino,ensino médio,solteiro,43,-1,-1,-1,-1,0,...,3502,8979,5190,0,1837,3526,8998,129,0,0
29997,30000,masculino,universidade,solteiro,37,4,3,2,-1,0,...,2758,20878,20582,19357,0,0,22000,4200,2000,3100
29998,80000,masculino,ensino médio,casado,41,1,-1,0,0,0,...,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804
